<H3>Условие</H3>
<ul>
    <li>Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 заданий в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.
</ul>   

<H3>Задача</H3>

<ul>
    <li>Даны таблицы: peas, studs, final_project_check</li>
</ul>

<p><b>Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:</b>
<ul>
    <li>ARPU</li>
    <li>ARPAU</li>
    <li>CR в покупку</li>
    <li>СR активного пользователя в покупку</li>
    <li>CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике</li>
    <li>ARPU считается относительно всех пользователей, попавших в группы.</li>
</ul>
<p>Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.
<p>Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [1]:
#импортируем библиотеки
import pandas as pd
import pandas.io.sql

from sqlite3 import connect

In [2]:
#считываем файлы в переменные
df_peas = pd.read_csv('peas.csv', 
                      sep=',', 
                      index_col=False, 
                      parse_dates = ['timest'], 
                      dtype={'st_id' : 'string', 
                             'correct' : 'int64',
                             'subject' : 'string'})

df_studs = pd.read_csv('studs.csv', 
                       sep=',', 
                       index_col=False,
                       dtype={'st_id' : 'string', 
                              'test_grp' : 'string'})

df_final_project_check = pd.read_csv('final_project_check.csv', 
                                     sep=',', 
                                     index_col=False,
                                     parse_dates = ['sale_time'],
                                     dtype={'st_id' : 'string', 
                                            'money' : 'float64',
                                            'subject' : 'string'})

In [3]:
df_studs

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot
3,104885,pilot
4,104966,pilot
...,...,...
595,988626,pilot
596,993249,pilot
597,994983,pilot
598,996911,control


In [4]:
df_peas

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability
3,100379,2021-10-30 16:44:50,1,Vizualization
4,100379,2021-10-30 17:15:05,1,Theory of probability
...,...,...,...,...
13240,996911,2021-10-31 08:41:00,1,Statistics
13241,996911,2021-10-31 09:12:28,0,Statistics
13242,996911,2021-10-31 09:33:39,0,Statistics
13243,996911,2021-10-31 09:46:44,1,Statistics


In [5]:
df_final_project_check

,st_id,sale_time,money,subject
0,101432,2021-10-31 04:44:32,85000.0,Math
1,101432,2021-10-31 12:43:50,65000.0,Vizualization
2,104885,2021-10-30 17:05:55,65000.0,Vizualization
3,104885,2021-10-30 22:49:33,75000.0,Statistics
4,106464,2021-10-31 13:17:13,85000.0,Math
...,...,...,...,...
56,969595,2021-10-30 23:54:52,65000.0,Vizualization
57,969595,2021-10-31 02:02:07,75000.0,Statistics
58,969595,2021-10-31 10:07:04,100000.0,Python
59,976866,2021-10-30 16:20:17,85000.0,Math


In [6]:
#содаем соединение с БД
conn = connect(':memory:')

df_peas.to_sql('peas', conn)
df_studs.to_sql('studs', conn)
df_final_project_check.to_sql('final_project_check', conn)

61

In [7]:
#пишем запрос к базе данных
query = '''
        SELECT
            test_grp,
            ROUND(SUM(is_client) / COUNT(DISTINCT user) * SUM(all_money) / SUM(is_client), 2) AS ARPU,
            ROUND(SUM(all_money) / SUM(is_active_user), 2) AS ARPPU,
            ROUND(SUM(all_money * is_active_user) / SUM(is_active_user), 2) AS ARPAU,
            ROUND(SUM(is_client) / COUNT(DISTINCT user) * 100, 2) AS CR,
            ROUND(SUM(is_client * is_active_user) / SUM(is_active_user) * 100, 2) AS active_CR,
            ROUND(SUM(is_math_client * is_math_active_user) / SUM(is_math_active_user) * 100, 2) AS math_CR
        FROM
            (
            SELECT 
                s.test_grp as test_grp,
                s.st_id AS user,
                CAST(COALESCE(SUM(all_money), 0) AS FLOAT) AS all_money,
                CAST(COALESCE(MAX(is_client), 0) AS FLOAT) AS is_client,
                CAST(COALESCE(MAX(is_math_client), 0) AS FLOAT) AS is_math_client,
                CAST(COALESCE(MAX(is_active_user), 0) AS FLOAT) AS is_active_user, 
                CAST(COALESCE(MAX(is_math_active_user), 0) AS FLOAT) AS is_math_active_user
            FROM studs AS s
            LEFT JOIN (
                SELECT 
                    st_id AS client,
                    1 AS is_client,
                    money AS all_money,
                    CASE WHEN subject = 'Math' THEN
                        1
                    ELSE
                        0
                    END AS is_math_client
                FROM final_project_check) AS c
            ON s.st_id = c.client
            LEFT JOIN (
                SELECT
                    st_id AS active_user,
                    1 AS is_active_user
                FROM peas
                GROUP BY st_id
                HAVING SUM(correct) > 10) AS active_users
            ON s.st_id = active_users.active_user
            LEFT JOIN (
                SELECT
                    st_id AS math_active_user,
                    1 AS is_math_active_user
                FROM peas
                WHERE subject = 'Math'
                GROUP BY st_id
                HAVING SUM(correct) > 1) AS active_maths
            ON s.st_id = active_maths.math_active_user
            GROUP BY test_grp, user
        ) AS t
        GROUP BY test_grp
        '''

In [8]:
pd.read_sql(query, conn)

,test_grp,ARPU,ARPPU,ARPAU,CR,active_CR,math_CR
0,control,4540.98,10905.51,10393.70,4.92,11.02,6.12
1,pilot,11508.47,35364.58,29739.58,10.85,26.04,9.52


<H3>Пояснение по блокам запроса</H3>

<p>Разберем каждый подзапрос в отдельности

In [9]:
#отфильтровываем тех пользователей, которые решили две и более задач по математике,
#а также создаем колонку is_math_active_user, где присваиваем значение 1 как маркер того, 
#что это активные пользователи по математике
query = '''
            SELECT
                st_id AS math_active_user,
                1 AS is_math_active_user
            FROM peas
            WHERE subject = 'Math'
            GROUP BY st_id
            HAVING SUM(correct) > 1
        '''
pd.read_sql(query, conn)

,math_active_user,is_math_active_user
0,106464,1
1,111336,1
2,128371,1
3,139814,1
4,143303,1
...,...,...
86,976866,1
87,985824,1
88,98637,1
89,988102,1


In [10]:
#отфильтровываем тех пользователей, которые решили более десяти задач в любых дисциплинах,
#а также создаем колонку is_active_user, где присваиваем значение 1 как маркер того, 
#что это активные пользователи
query = '''
            SELECT
                st_id AS active_user,
                1 AS is_active_user
            FROM peas
            GROUP BY st_id
            HAVING SUM(correct) > 10
        '''
pd.read_sql(query, conn)

,active_user,is_active_user
0,100379,1
1,101432,1
2,104966,1
3,106464,1
4,111336,1
...,...,...
218,983096,1
219,98637,1
220,986680,1
221,994983,1


In [11]:
#отфильтровываем тех пользователей, которые купили курсы
#создаем колонку is_client с маркером того, что эти пользователя являются покупателями
#создаем колонку is_math_client, где при условии, что subject = 'Math' это покупатель курсов по математике
#колонка all_money содержит стоимость курса отдельной дисциплины
query = '''
            SELECT 
                st_id AS client,
                1 AS is_client,
                money AS all_money,
                CASE WHEN subject = 'Math' THEN
                    1
                ELSE
                    0
                END AS is_math_client
            FROM final_project_check
        '''
pd.read_sql(query, conn)

,client,is_client,all_money,is_math_client
0,101432,1,85000.0,1
1,101432,1,65000.0,0
2,104885,1,65000.0,0
3,104885,1,75000.0,0
4,106464,1,85000.0,1
...,...,...,...,...
56,969595,1,65000.0,0
57,969595,1,75000.0,0
58,969595,1,100000.0,0
59,976866,1,85000.0,1


<b>Далее левым соединением объединяем три предыдущих запроса вместе</b>


<ul>
    <li>Назначение функции MAX: т.к. в подзапросах мы присваивали маркеры со значением 1, то теперь нам надо вытащить эти маркеры в групприовку. Но те записи, которые не содержат значение 1, будут выдавать значение NULL, а нам это не надо. Поэтому мы скажем на языке SQL при помощи функции COALESCE, что если значение в колонке отличается от типа "число" (в нашем случае), то присвой значение 0.</li>
    <li>Назначение функции SUM: мы просто суммируем колонку all_money, чтобы записи с одинковым ИД пользователя схлопнулись, а на выходе мы получили общую сумму его покупок.</li>
    <li>Назначение функции CAST: предназначена для того, чтобы привести колонки к типу FLOAT, иначе у нас ничего не посчитается при финальном расчете метрик.</li>
</ul>

In [12]:

query = '''
            SELECT 
                s.test_grp as test_grp,
                s.st_id AS user,
                CAST(COALESCE(SUM(all_money), 0) AS FLOAT) AS all_money,
                CAST(COALESCE(MAX(is_client), 0) AS FLOAT) AS is_client,
                CAST(COALESCE(MAX(is_math_client), 0) AS FLOAT) AS is_math_client,
                CAST(COALESCE(MAX(is_active_user), 0) AS FLOAT) AS is_active_user, 
                CAST(COALESCE(MAX(is_math_active_user), 0) AS FLOAT) AS is_math_active_user
            FROM studs AS s
            LEFT JOIN (
                SELECT 
                    st_id AS client,
                    1 AS is_client,
                    money AS all_money,
                    CASE WHEN subject = 'Math' THEN
                        1
                    ELSE
                        0
                    END AS is_math_client
                FROM final_project_check) AS c
            ON s.st_id = c.client
            LEFT JOIN (
                SELECT
                    st_id AS active_user,
                    1 AS is_active_user
                FROM peas
                GROUP BY st_id
                HAVING SUM(correct) > 10) AS active_users
            ON s.st_id = active_users.active_user
            LEFT JOIN (
                SELECT
                    st_id AS math_active_user,
                    1 AS is_math_active_user
                FROM peas
                WHERE subject = 'Math'
                GROUP BY st_id
                HAVING SUM(correct) > 1) AS active_maths
            ON s.st_id = active_maths.math_active_user
            GROUP BY test_grp, user
        '''
pd.read_sql(query, conn)

,test_grp,user,all_money,is_client,is_math_client,is_active_user,is_math_active_user
0,control,101432,150000.0,1.0,1.0,1.0,0.0
1,control,106816,0.0,0.0,0.0,0.0,0.0
2,control,107250,0.0,0.0,0.0,0.0,0.0
3,control,111336,0.0,0.0,0.0,1.0,1.0
4,control,11503,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
595,pilot,986894,0.0,0.0,0.0,0.0,0.0
596,pilot,988626,0.0,0.0,0.0,0.0,0.0
597,pilot,993249,0.0,0.0,0.0,0.0,0.0
598,pilot,994983,0.0,0.0,0.0,1.0,1.0


<b>На выходе получаем такую таблицу, где у нас для каждого пользователя есть отдельная роль (наши маркеры)</b>
<hr>

<p>А дальше мы просто считаем метрики по формулам</p>
<b>Но отдельно хочу пояснить, что происходит в следующих агрегациях:</b>
<ul>
    <li>SUM(is_client * is_active_user)</li>
    <li>SUM(is_math_client * is_math_active_user)</li>
</ul>

<p>Т.к. нам нужно опеределить то, является ли покупатель курса  активным пользователем курса и наоборот, то мы просто используем математический подход, при котором если, например, is_client = 0, а is_active_user = 1, то при умножении мы получим 0. В таком случае этот пользователь будет являться активным пользователем курса, но не покупателем этого курса, и, значит, он не попадет в подсчет количества активных пользователей, которые купили курс. А дальше при помощи функции SUM() мы считаем количество пользователей.

In [13]:
#пишем запрос к базе данных
query = '''
        SELECT
            test_grp,
            ROUND(SUM(is_client) / COUNT(DISTINCT user) * SUM(all_money) / SUM(is_client), 2) AS ARPU,
            ROUND(SUM(all_money) / SUM(is_active_user), 2) AS ARPPU,
            ROUND(SUM(all_money * is_active_user) / SUM(is_active_user), 2) AS ARPAU,
            ROUND(SUM(is_client) / COUNT(DISTINCT user) * 100,2) AS CR,
            ROUND(SUM(is_client * is_active_user) / SUM(is_active_user) * 100, 2) AS active_CR,
            ROUND(SUM(is_math_client * is_math_active_user) / SUM(is_math_active_user) * 100, 2) AS math_CR
        FROM
            (
            SELECT 
                s.test_grp as test_grp,
                s.st_id AS user,
                CAST(COALESCE(SUM(all_money), 0) AS FLOAT) AS all_money,
                CAST(COALESCE(MAX(is_client), 0) AS FLOAT) AS is_client,
                CAST(COALESCE(MAX(is_math_client), 0) AS FLOAT) AS is_math_client,
                CAST(COALESCE(MAX(is_active_user), 0) AS FLOAT) AS is_active_user, 
                CAST(COALESCE(MAX(is_math_active_user), 0) AS FLOAT) AS is_math_active_user
            FROM studs AS s
            LEFT JOIN (
                SELECT 
                    st_id AS client,
                    1 AS is_client,
                    money AS all_money,
                    CASE WHEN subject = 'Math' THEN
                        1
                    ELSE
                        0
                    END AS is_math_client
                FROM final_project_check) AS c
            ON s.st_id = c.client
            LEFT JOIN (
                SELECT
                    st_id AS active_user,
                    1 AS is_active_user
                FROM peas
                GROUP BY st_id
                HAVING SUM(correct) > 10) AS active_users
            ON s.st_id = active_users.active_user
            LEFT JOIN (
                SELECT
                    st_id AS math_active_user,
                    1 AS is_math_active_user
                FROM peas
                WHERE subject = 'Math'
                GROUP BY st_id
                HAVING SUM(correct) > 1) AS active_maths
            ON s.st_id = active_maths.math_active_user
            GROUP BY test_grp, user
        ) AS t
        GROUP BY test_grp
        '''
pd.read_sql(query, conn)

,test_grp,ARPU,ARPPU,ARPAU,CR,active_CR,math_CR
0,control,4540.98,10905.51,10393.70,4.92,11.02,6.12
1,pilot,11508.47,35364.58,29739.58,10.85,26.04,9.52
